<a href="https://colab.research.google.com/github/vmartinezalvarez/Image-generation-GAN-vs-DCGAN/blob/master/Project_NotMNIST_DCGAN_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import  libraries

In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2DTranspose, Conv2D, UpSampling2D
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
# from keras.layers.convolutional import UpSampling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop 
from keras.preprocessing import image

import os
from skimage import io
from skimage.transform import resize
import tarfile
import time
from IPython import display

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [2]:
! pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=8121917994edde1ffa29e110250eb4f1db9e38c23e9d73b113b2006951a4a219
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


## notMNIST dataset
 The [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset is similar to MNIST, but looks a bit more like real data, that is, the data is much less clean compared to MNIST. 
 
There are 10 classes, with letters A-J taken from different fonts. 

The dataset consists of a small part cleaned by hand, approximately 1872 elements per class, and large uncleaned part, approximately 52909 elements per class. Two parts have approximately 0.5% and 6.5% label error rate.


#### Download notMNIST_large and notMNIST_small dataset

In [3]:
import wget

file_url_large = 'https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz'
file_url_small = 'https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz'

file_name_large = wget.download(file_url_large)
file_name_small = wget.download(file_url_small)

In [4]:
print(file_name_large)
print(file_name_small)

notMNIST_large.tar.gz
notMNIST_small.tar.gz


In [5]:
start = time.time()

# extract notMNIST_large
extract_large = tarfile.open(file_name_large)
extract_large.extractall()

# extract notMNIST_small
extract_small = tarfile.open(file_name_small)
extract_small.extractall()
print ('Time to extract is {} sec'.format(time.time()-start))

Time to extract is 112.4398398399353 sec


### Create the dataset to traing the DCGAN model

The code below is intended to be used to generate a dataset with all the samples together, that is, a mix of the clean and unclean parts.

In [ ]:
#Training data
train = []
train_labels = []

start = time.time()
################### Letter A ######################### 

files = io.ImageCollection('notMNIST_large/A' + '/*.png') #  image path
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

files = io.ImageCollection('notMNIST_small/A' + '/*.png') 
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

################### Letter B ######################### 

files =  io.ImageCollection('notMNIST_large/B' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
    
files =  io.ImageCollection('notMNIST_small/B' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

################### Letter C ######################### 

files =  io.ImageCollection('notMNIST_large/C' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

files =  io.ImageCollection('notMNIST_small/C' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

################### Letter D ######################### 

files =  io.ImageCollection('notMNIST_large/D' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]) 
    
files =  io.ImageCollection('notMNIST_small/D' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
    
 ################### Letter E ######################### 

files =  io.ImageCollection('notMNIST_large/E' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
    
    
files =  io.ImageCollection('notMNIST_small/E' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

################### Letter F ######################### 

files =  io.ImageCollection('notMNIST_large/F' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

files =  io.ImageCollection('notMNIST_small/F' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
    
    
################### Letter G #########################   
files =  io.ImageCollection('notMNIST_large/G' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
    
files =  io.ImageCollection('notMNIST_small/G' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
    
################### Letter H  #########################   
files =  io.ImageCollection('notMNIST_large/H' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

files =  io.ImageCollection('notMNIST_small/H' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])
    
    
################### Letter I  #########################   
files =  io.ImageCollection('notMNIST_large/I' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

files =  io.ImageCollection('notMNIST_small/I' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
    

################### Letter J   #########################   
files =  io.ImageCollection('notMNIST_large/J' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
    
files =  io.ImageCollection('notMNIST_small/J' + '/*.png')
for myFile in files:
    train.append(resize(myFile,(28,28,1)))
    train_labels.append([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])   
    
print ('Time to create dataset is {} sec'.format(time.time()-start))

### Create the dataset with only the clean part

The code below is intended to be used to generate a dataset with all the samples in the clean part.

In [ ]:
#Training data
train_clean = []
train_labels_clean = []

start = time.time()
################### Letter A ######################### 

files = io.ImageCollection('notMNIST_small/A' + '/*.png') 
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

################### Letter B ######################### 

files =  io.ImageCollection('notMNIST_small/B' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

################### Letter C ######################### 

files =  io.ImageCollection('notMNIST_small/C' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

################### Letter D ######################### 

files =  io.ImageCollection('notMNIST_small/D' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
    
 ################### Letter E ######################### 

files =  io.ImageCollection('notMNIST_small/E' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

################### Letter F ######################### 

files =  io.ImageCollection('notMNIST_small/F' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
    
    
################### Letter G #########################   

files =  io.ImageCollection('notMNIST_small/G' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
    
################### Letter H  #########################   

files =  io.ImageCollection('notMNIST_small/H' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])
    
    
################### Letter I  #########################   

files =  io.ImageCollection('notMNIST_small/I' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
    

################### Letter J   #########################   
    
files =  io.ImageCollection('notMNIST_small/J' + '/*.png')
for myFile in files:
    train_clean.append(resize(myFile,(28,28,1)))
    train_labels_clean.append([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])   
    
print ('Time to create dataset is {} sec'.format(time.time()-start))

#### Create the compressed dataset 

In [ ]:
# save numpy array as .npy formats
np.save('train',train)
np.save('train_labels',train_labels)
np.savez_compressed('dataset', x_train=train, y_train=train_labels)

#### Create the compressed clean dataset  

In [ ]:
# save numpy array as .npy formats
np.save('train_clean',train_clean)
np.save('train_labels_clean',train_labels_clean)
np.savez_compressed('dataset_clean', x_train_clean=train_clean, y_train_clean=train_labels_clean)

### Once created the dataset, download it to the hard disk and then upload it to google drive RIST, which is the destination folder for this project.

#### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! ls "/content/drive/My Drive/Project"

In [ ]:
def load_data(path):
    with np.load(path) as f:
        x_train, y_train = f['x_train_clean'], f['y_train_clean']
        return (x_train, y_train)

      
my_path       = "/content/drive/My Drive/Project/dataset.npz"
my_path_clean = "/content/drive/My Drive/Project/dataset_clean.npz"
      
(x_train_clean, y_train_clean) = load_data(my_path_clean)

In [ ]:
# number of samples
x_train_clean.shape[0]

In [ ]:
x_train_clean.shape

In [ ]:
# number of training samples for classes in the dataset
y_train_clean.sum(axis=0)

As we can see, the dataset is well balanced across classes

#### Bellow we can see some examples of letter "B" 

In [ ]:
fig = plt.figure(figsize=(8,8))
  
idxs = np.random.randint(1872, 1872 + 500, 25)
images = x_train_clean[idxs]
  
for i in range(images.shape[0]):
  plt.subplot(5, 5, i+1)
  plt.imshow(images[i, :, :, 0], cmap='gray')
  plt.axis('off')     
plt.show()

#### Show random samples

In [ ]:
fig = plt.figure(figsize=(8,8))
  
idxs = np.random.randint(0, x_train_clean.shape[0], 25)
images = x_train_clean[idxs]
  
for i in range(images.shape[0]):
  plt.subplot(5, 5, i+1)
  plt.imshow(images[i, :, :, 0], cmap='gray')
  plt.axis('off')     
plt.show()

## Implement the DCGAN class

In [ ]:
class NotMNIST_DCGAN_1():
  def __init__(self, height, width, channels, z_dim_noise):
    
    # input noise dimension
    self.z_dim = z_dim_noise
    # Input shape   
    self.img_height   = height          
    self.img_width    = width           
    self.img_channels = channels        
    self.img_shape    = (self.img_height, 
                         self.img_width,
                         self.img_channels)
    


    # Create the discriminator
    self.discriminator = self.discriminator_model()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0003, 0.7), metrics=['accuracy'])

    # Create the generator
    self.generator = self.generator_model()
    
    # Create the gan 
    self.gan = self.gan_model()
    self.gan.compile(loss='binary_crossentropy', optimizer = Adam(0.0003, 0.7))  
    
    
################################# gan model ###################################################
  def gan_model(self):
    
    # input to gan
    gan_input = Input(shape=(self.z_dim,))
    
    # generated fake images
    generated_images = self.generator(gan_input)
    
    # freeze the discriminator
    self.discriminator.trainable = False
    
    # Trains the generator to fool the discriminator
    gan_output = self.discriminator(generated_images)
    
    return Model(gan_input, gan_output)

################################# generator model ###################################################
  def generator_model(self):
    # input to generator
    input_noise = Input(shape=(self.z_dim,))

    # define sequential model 
    model = Sequential()    
    # 1st  layer  
    model.add(Dense(7*7*512, use_bias=False, activation=None, input_dim=self.z_dim ))
    model.add(Reshape((7, 7, 512)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())  
    # print(model.output_shape)
    # 2nd  layer
    model.add(Conv2DTranspose(256, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())  
    # print(model.output_shape)
    # 3rd layer      
    model.add(Conv2DTranspose(128, kernel_size=(5,5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    # print(model.output_shape)
    # 4th layer
    model.add(Conv2DTranspose(64, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    # print(model.output_shape)
    
    # Output layer
    model.add(Conv2DTranspose(1, kernel_size=(5,5), strides=(2, 2), padding='same', 
                              use_bias=False, activation='tanh'))
    
    #print(model.output_shape)
    #assert model.output_shape == (None, 28, 28, 1)
    model.summary()
    generator_output = model(input_noise)

    return Model(input_noise, generator_output)

#   def generator_model(self):
#     # input to generator
#     input_noise = Input(shape=(self.z_dim,))

#     # define sequential model 
#     model = Sequential()
#     # 1st  layer
#     model.add(Dense(7*7*256, use_bias=False, activation=None, input_dim=self.z_dim ))
#     model.add(Reshape((7, 7, 256)))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU())
#     # print(model.output_shape)      

#     # 2nd layer      
#     model.add(Conv2DTranspose(128, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU()) 
#     # print(model.output_shape)
#     # 3rd layer
#     model.add(Conv2DTranspose(64, kernel_size=(3,3), strides=(2, 2), padding='same', use_bias=False))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU()) 
#     print(model.output_shape)
#     # 4th layer
#     model.add(Conv2DTranspose(32, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU()) 
#     print(model.output_shape)
    
#     # Output layer
#     model.add(Conv2DTranspose(1, kernel_size=(5,5), strides=(2, 2), padding='same', 
#                               use_bias=False, activation='tanh'))
    
# #     print(model.output_shape)
# #     assert model.output_shape == (None, 28, 28, 1)
#     model.summary()
#     generator_output = model(input_noise)

#     return Model(input_noise, generator_output)

################################# discriminator model ###################################################

  def discriminator_model(self):
    
    # input to discriminator
    input_img = Input(shape=self.img_shape)
    
    # define sequential model
    model = Sequential()
    # 1st  layer
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=self.img_shape))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    # print(model.output_shape)
    # 2nd  layer
    model.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    # 3rd  layer
    model.add(Conv2D(256, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    #print(model.output_shape)
    #4th  layer
    model.add(Conv2D(512, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    #print(model.output_shape)
    #Output layer   
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    discriminator_output = model(input_img)
    
    return Model(input_img, discriminator_output)
  
#################################### train method ################################################
  
  def train(self, epochs, train_data, batch_size=256, log_inter=50):
    # shuffle dataset
    train_data = self.randomize(train_data)
    # Reshape it    
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1).astype('float32')
    train_data = (train_data - 0.5) / 0.5 # Normalize the images to [-1, 1]
    # losses of Generator and Discriminator
    losses = {"Dis":[], "Gen":[]} 
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))


    for epoch in range(epochs):

      #  Train Discriminator
      # Select a batch of random  images
      index = np.random.randint(0, train_data.shape[0], batch_size)
      imgs = train_data[index]

      # Sample noise and generate a batch of fake images
      noise = np.random.normal(0, 1, (batch_size, self.z_dim))
      gen_imgs = self.generator.predict(noise)

      # important trick: add random noise to the labels
      # Discriminator loss
      d_loss_real = self.discriminator.train_on_batch(imgs, real+ 0.05*np.random.random(real.shape))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake +0.05*np.random.random(fake.shape))
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      losses["Dis"].append(d_loss[0])
      

      #  train Generator via gan 
      g_loss = self.gan.train_on_batch(noise, real)
      losses["Gen"].append(g_loss)
      
      
      # If at save interval => save generated image samples
      if epoch % log_inter == 0:
        display.clear_output(wait=True)
        self.generate_and_save_images(epoch)
        self.plot_loss(losses)
        #print('Epoch:{} [Discriminative loss: {:.4f}, accuracy: {:.2f}%],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], 100*d_loss[1], g_loss))  
        print('Epoch:{} [Discriminative loss: {:.4f}],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], g_loss))  
        

        
    display.clear_output(wait=True)
    self.generate_and_save_images(epoch)
    self.plot_loss(losses)
      
      
  def generate_and_save_images(self, epoch):
    # training is set to False, all layers run in inference mode
    noise = np.random.normal(0, 1, (25, self.z_dim))
    predictions =  self.generator.predict(noise)

    fig = plt.figure(figsize=(5,5))

    for i in range(predictions.shape[0]):
      plt.subplot(5, 5, i+1)
      plt.imshow(predictions[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
      plt.axis('off')

    label = 'Epoch {0}'.format(epoch)
    fig.text(0.51, 0.05, label, ha='center',fontsize=14)      
    #plt.savefig('NotMNIST_DCGAN_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

  def plot_loss(self, losses):
    plt.figure(figsize=(6,5))
    plt.plot(losses["Dis"], label='discrimininator loss')
    plt.plot(losses["Gen"], label='generator loss')
    plt.legend()
    plt.show()
    
  def randomize(self, dataset):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    return shuffled_dataset

In [ ]:
imagen = image.array_to_img(x_train_clean[12], scale=False)
plt.figure()
plt.imshow(imagen,cmap='gray')    
plt.show()

In [ ]:
height   = x_train_clean.shape[1]
width    = x_train_clean.shape[2]
channels = x_train_clean.shape[3]
z_dim_noise = 100

In [ ]:
# Important, this dataset is between o and 1,
print(np.min(x_train_clean), np.max(x_train_clean))

In [ ]:
train_data_test = (x_train_clean - 0.5) / 0.5 # Normalize the images to [-1, 1]
print(np.min(train_data_test), np.max(train_data_test))

#### Create the object 

In [ ]:
!mkdir -p NotMNIST_DCGAN_images

In [ ]:
dcgan = NotMNIST_DCGAN(height, width, channels, z_dim_noise)
dcgan.train(epochs=1000, train_data = x_train_clean, batch_size=256, log_inter=50)

### Second Model

In [ ]:
class NotMNIST_DCGAN_2():
  def __init__(self, height, width, channels, z_dim_noise):
    # input noise dimension
    self.z_dim = z_dim_noise
    # Input shape   
    self.img_height   = height          
    self.img_width    = width           
    self.img_channels = channels        
    self.img_shape    = (self.img_height, 
                         self.img_width,
                         self.img_channels)
    

    
    # Create the discriminator
    self.discriminator = self.discriminator_model()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, beta_1=0.99), metrics=['accuracy'])

    # Create the generator
    self.generator = self.generator_model()
    
    # Create the gan 
    self.gan = self.gan_model()
    self.gan.compile(loss='binary_crossentropy', optimizer = Adam(lr=0.0001, beta_1=0.99))  
    


################################# generator model ###################################################
  def generator_model(self):
    # input to generator
    input_noise = Input(shape=(self.z_dim,))

    # define sequential model 
    model = Sequential()
    # 1st  layer
    model.add(Dense(7*7*256, use_bias=False, activation=None, input_dim=self.z_dim ))
    model.add(Reshape((7, 7, 256)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    # print(model.output_shape)      

    # 2nd layer      
    model.add(Conv2DTranspose(128, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    # print(model.output_shape)
    # 3rd layer
    model.add(Conv2DTranspose(64, kernel_size=(3,3), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    print(model.output_shape)
    # 4th layer
    model.add(Conv2DTranspose(32, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    print(model.output_shape)
    
    # Output layer
    model.add(Conv2DTranspose(1, kernel_size=(5,5), strides=(2, 2), padding='same', 
                              use_bias=False, activation='tanh'))
    
#     print(model.output_shape)
#     assert model.output_shape == (None, 28, 28, 1)
    model.summary()
    generator_output = model(input_noise)

    return Model(input_noise, generator_output)

################################# discriminator model ###################################################

  def discriminator_model(self):
    
    # input to discriminator
    input_img = Input(shape=self.img_shape)
    
    # define sequential model
    model = Sequential()
    # 1st  layer
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=self.img_shape))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    # print(model.output_shape)
    # 2nd  layer
    model.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    # 3rd  layer
    model.add(Conv2D(256, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
#     #print(model.output_shape)
#     #4th  layer
#     model.add(Conv2D(512, kernel_size=(5, 5), strides=(2, 2), padding='same'))
#     model.add(LeakyReLU())
#     model.add(Dropout(0.3))
#     #print(model.output_shape)
#     #Output layer   
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    discriminator_output = model(input_img)
    
    return Model(input_img, discriminator_output)
      
################################# gan model ###################################################
  def gan_model(self):
    
    # input to gan
    gan_input = Input(shape=(self.z_dim,))
    
    # generated fake images
    generated_images = self.generator(gan_input)
    
    # freeze the discriminator
    self.discriminator.trainable = False
    
    # Trains the generator to fool the discriminator
    gan_output = self.discriminator(generated_images)
    
    return Model(gan_input, gan_output)
  
#################################### train method ################################################
  
  def train(self, epochs, train_data, batch_size=256, log_inter=50):
    # shuffle dataset
    #train_data = self.randomize(train_data)
    # Reshape it    
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1).astype('float32')
    train_data = (train_data - 0.5) / 0.5 # Normalize the images to [-1, 1]
    # losses of Generator and Discriminator
    losses = {"Dis":[], "Gen":[]} 
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    inicial = 0

    for epoch in range(epochs):
      
      final = inicial + batch_size
      imgs = train_data[inicial: final]
      
      #  Train Discriminator
      # Select a batch of random  images
#       index = np.random.randint(0, train_data.shape[0], batch_size)
#       imgs = train_data[index]
     
      # Sample noise and generate a batch of fake images
      noise = np.random.normal(0, 1, (batch_size, self.z_dim))
      gen_imgs = self.generator.predict(noise)

      # important trick: add random noise to the labels
      # Discriminator loss
      d_loss_real = self.discriminator.train_on_batch(imgs, fake + 0.07*np.random.random(fake.shape))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, real -0.07*np.random.random(real.shape))
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      losses["Dis"].append(d_loss[0])
      

      #  train Generator via gan 
      g_loss = self.gan.train_on_batch(noise, fake)
      losses["Gen"].append(g_loss)
      
      
      # If at save interval => save generated image samples
      if epoch % log_inter == 0:
        display.clear_output(wait=True)
        self.generate_and_save_images(epoch)
        self.plot_loss(losses)
        #print('Epoch:{} [Discriminative loss: {:.4f}, accuracy: {:.2f}%],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], 100*d_loss[1], g_loss))  
        print('Epoch:{} [Discriminative loss: {:.4f}],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], g_loss))  
      
      inicial += batch_size
      if (inicial > len(train_data) - batch_size):
        inicial = 0  

        
    display.clear_output(wait=True)
    self.generate_and_save_images(epoch)
    self.plot_loss(losses)
      
      
  def generate_and_save_images(self, epoch):
    # training is set to False, all layers run in inference mode
    noise = np.random.normal(0, 1, (25, self.z_dim))
    predictions =  self.generator.predict(noise)

    fig = plt.figure(figsize=(5,5))

    for i in range(predictions.shape[0]):
      plt.subplot(5, 5, i+1)
      plt.imshow(predictions[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
      plt.axis('off')

    label = 'Epoch {0}'.format(epoch)
    fig.text(0.51, 0.05, label, ha='center',fontsize=14)      
    plt.savefig('NotMNIST_DCGAN_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

  def plot_loss(self, losses):
    plt.figure(figsize=(6,5))
    plt.plot(losses["Dis"], label='discrimininator loss')
    plt.plot(losses["Gen"], label='generator loss')
    plt.legend()
    plt.show()
    
  def randomize(self, dataset):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    return shuffled_dataset

In [ ]:
dcgan_2 = NotMNIST_DCGAN_2(height, width, channels, z_dim_noise)
dcgan_2.train(epochs=8000, train_data = x_train_clean, batch_size=256, log_inter=50)

In [ ]:
import glob
import imageio
with imageio.get_writer('NotMNIST.gif', mode='I') as writer:
  filenames = glob.glob('NotMNIST_DCGAN_images/image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# A hack to display the GIF inside this notebook
os.rename('NotMNIST.gif', 'NotMNIST.gif.png')

In [ ]:
display.Image(filename="NotMNIST.gif.png")

In [ ]:
class NotMNIST_DCGAN_3():
  def __init__(self, height, width, channels, z_dim_noise):
    # input noise dimension
    self.z_dim = z_dim_noise
    # Input shape   
    self.img_height   = height          
    self.img_width    = width           
    self.img_channels = channels        
    self.img_shape    = (self.img_height, 
                         self.img_width,
                         self.img_channels)
    

    
    # Create the discriminator
    self.discriminator = self.discriminator_model()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005, beta_1=0.9), metrics=['accuracy'])

    # Create the generator
    self.generator = self.generator_model()
    
    # Create the gan 
    self.gan = self.gan_model()
    self.gan.compile(loss='binary_crossentropy', optimizer = Adam(lr=0.0005, beta_1=0.91))  
    


################################# generator model ###################################################
  def generator_model(self):
    # input to generator
    input_noise = Input(shape=(self.z_dim,))

    # define sequential model 
    model = Sequential()
    # 1st  layer
    model.add(Dense(14*14*128, use_bias=False, activation=None, input_dim=self.z_dim ))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Reshape((14, 14, 128)))
    print(model.output_shape)      

    # 2nd layer      
    model.add(Conv2D(128, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    # print(model.output_shape)
    model.add(Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    # 3rd layer
    model.add(Conv2DTranspose(128, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    print(model.output_shape)
    
    # Output layer
    model.add(Conv2D(1, kernel_size=(7,7), strides=(1, 1), padding='same', 
                              use_bias=False, activation='tanh'))
    
    print(model.output_shape)
    assert model.output_shape == (None, 28, 28, 1)
    model.summary()
    generator_output = model(input_noise)

    return Model(input_noise, generator_output)

################################# discriminator model ###################################################

  def discriminator_model(self):
    
    # input to discriminator
    input_img = Input(shape=self.img_shape)
    
    # define sequential model
    model = Sequential()
    # 1st  layer
    model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=self.img_shape))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    # print(model.output_shape)
    # 2nd  layer
    model.add(Conv2D(128, kernel_size=(4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    # 3rd  layer
    model.add(Conv2D(128, kernel_size=(4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    # 4th  layer
    model.add(Conv2D(64, kernel_size=(4, 4), strides=(2, 2),  padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    # print(model.output_shape)
    # Output layer
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    discriminator_output = model(input_img)
    
    return Model(input_img, discriminator_output)
      
################################# gan model ###################################################
  def gan_model(self):
    
    # input to gan
    gan_input = Input(shape=(self.z_dim,))
    
    # generated fake images
    generated_images = self.generator(gan_input)
    
    # freeze the discriminator
    self.discriminator.trainable = False
    
    # Trains the generator to fool the discriminator
    gan_output = self.discriminator(generated_images)
    
    return Model(gan_input, gan_output)
  
#################################### train method ################################################
  
  def train(self, epochs, train_data, batch_size=256, log_inter=50):
    # shuffle dataset
    #train_data = self.randomize(train_data)
    # Reshape it    
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1).astype('float32')
    train_data = (train_data - 0.5) / 0.5 # Normalize the images to [-1, 1]
    # losses of Generator and Discriminator
    losses = {"Dis":[], "Gen":[]} 
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    inicial = 0

    for epoch in range(epochs):
      
      final = inicial + batch_size
      imgs = train_data[inicial: final]
      
      #  Train Discriminator
      # Select a batch of random  images
#       index = np.random.randint(0, train_data.shape[0], batch_size)
#       imgs = train_data[index]
     
      # Sample noise and generate a batch of fake images
      noise = np.random.normal(0, 1, (batch_size, self.z_dim))
      gen_imgs = self.generator.predict(noise)

      # important trick: add random noise to the labels
      # Discriminator loss
      d_loss_real = self.discriminator.train_on_batch(imgs, fake + 0.07*np.random.random(fake.shape))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, real -0.07*np.random.random(real.shape))
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      losses["Dis"].append(d_loss[0])
      

      #  train Generator via gan 
      g_loss = self.gan.train_on_batch(noise, fake)
      losses["Gen"].append(g_loss)
      
      
      # If at save interval => save generated image samples
      if epoch % log_inter == 0:
        display.clear_output(wait=True)
        self.generate_and_save_images(epoch)
        self.plot_loss(losses)
        #print('Epoch:{} [Discriminative loss: {:.4f}, accuracy: {:.2f}%],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], 100*d_loss[1], g_loss))  
        print('Epoch:{} [Discriminative loss: {:.4f}],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], g_loss))  
      
      inicial += batch_size
      if (inicial > len(train_data) - batch_size):
        inicial = 0  

        
    display.clear_output(wait=True)
    self.generate_and_save_images(epoch)
    self.plot_loss(losses)
      
      
  def generate_and_save_images(self, epoch):
    # training is set to False, all layers run in inference mode
    noise = np.random.normal(0, 1, (25, self.z_dim))
    predictions =  self.generator.predict(noise)

    fig = plt.figure(figsize=(5,5))

    for i in range(predictions.shape[0]):
      plt.subplot(5, 5, i+1)
      plt.imshow(predictions[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
      plt.axis('off')

    label = 'Epoch {0}'.format(epoch)
    fig.text(0.51, 0.05, label, ha='center',fontsize=14)      
    plt.savefig('NotMNIST_DCGAN_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

  def plot_loss(self, losses):
    plt.figure(figsize=(6,5))
    plt.plot(losses["Dis"], label='discrimininator loss')
    plt.plot(losses["Gen"], label='generator loss')
    plt.legend()
    plt.show()
    
  def randomize(self, dataset):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    return shuffled_dataset

In [ ]:
dcgan_3 = NotMNIST_DCGAN_3(height, width, channels, z_dim_noise)
dcgan_3.train(epochs=8000, train_data = x_train_clean, batch_size=256, log_inter=50)

## Model NotMNIST_DCGAN_4

In [ ]:
class NotMNIST_DCGAN_4():
  def __init__(self, height, width, channels, z_dim_noise):
    # input noise dimension
    self.z_dim = z_dim_noise
    # Input shape   
    self.img_height   = height          
    self.img_width    = width           
    self.img_channels = channels        
    self.img_shape    = (self.img_height, 
                         self.img_width,
                         self.img_channels)
    

    
    # Create the discriminator
    self.discriminator = self.discriminator_model()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0004, beta_1=0.9,decay=1e-8), metrics=['accuracy'])

    # Create the generator
    self.generator = self.generator_model()
    
    # Create the gan 
    self.gan = self.gan_model()
    self.gan.compile(loss='binary_crossentropy', optimizer = Adam(lr=0.0002, beta_1=0.9,decay=1e-8))  
    


################################# generator model ###################################################
  def generator_model(self):
    # input to generator
    input_noise = Input(shape=(self.z_dim,))

    # define sequential model 
    model = Sequential()
    # 1st  layer
    model.add(Dense(7*7*256, use_bias=False, activation=None, input_dim=self.z_dim ))
    model.add(Reshape((7, 7, 256)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    # print(model.output_shape)      

    # 2nd layer      
    model.add(Conv2DTranspose(128, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    # print(model.output_shape)
    # 3rd layer
    model.add(Conv2DTranspose(64, kernel_size=(3,3), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    print(model.output_shape)
    # 4th layer
    model.add(Conv2DTranspose(32, kernel_size=(5,5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) 
    print(model.output_shape)
    
    # Output layer
    model.add(Conv2DTranspose(1, kernel_size=(5,5), strides=(2, 2), padding='same', 
                              use_bias=False, activation='tanh'))
    
#     print(model.output_shape)
#     assert model.output_shape == (None, 28, 28, 1)
    model.summary()
    generator_output = model(input_noise)

    return Model(input_noise, generator_output)

################################# discriminator model ###################################################

  def discriminator_model(self):
    
    # input to discriminator
    input_img = Input(shape=self.img_shape)
    
    # define sequential model
    model = Sequential()
    # 1st  layer
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=self.img_shape))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    # print(model.output_shape)
    # 2nd  layer
    model.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    # 3rd  layer
    model.add(Conv2D(256, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.35))
#     #print(model.output_shape)
#     #Output layer   
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    discriminator_output = model(input_img)
    
    return Model(input_img, discriminator_output)
      
################################# gan model ###################################################
  def gan_model(self):
    
    # input to gan
    gan_input = Input(shape=(self.z_dim,))
    
    # generated fake images
    generated_images = self.generator(gan_input)
    
    # freeze the discriminator
    self.discriminator.trainable = False
    
    # Trains the generator to fool the discriminator
    gan_output = self.discriminator(generated_images)
    
    return Model(gan_input, gan_output)
  
#################################### train method ################################################
  
  def train(self, epochs, train_data, batch_size=256, log_inter=50):
    # shuffle dataset
    #train_data = self.randomize(train_data)
    # Reshape it    
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1).astype('float32')
    train_data = (train_data - 0.5) / 0.5 # Normalize the images to [-1, 1]
    # losses of Generator and Discriminator
    losses = {"Dis":[], "Gen":[]} 
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    inicial = 0

    for epoch in range(epochs):
      
      final = inicial + batch_size
      imgs = train_data[inicial: final]
      
      #  Train Discriminator
      # Select a batch of random  images
#       index = np.random.randint(0, train_data.shape[0], batch_size)
#       imgs = train_data[index]
     
      # Sample noise and generate a batch of fake images
      noise = np.random.normal(0, 1, (batch_size, self.z_dim))
      gen_imgs = self.generator.predict(noise)

      # important trick: add random noise to the labels
      # Discriminator loss
      d_loss_real = self.discriminator.train_on_batch(imgs, fake + 0.05*np.random.random(fake.shape))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, real -0.05*np.random.random(real.shape))
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      losses["Dis"].append(d_loss[0])
      

      #  train Generator via gan 
      g_loss = self.gan.train_on_batch(noise, fake)
      losses["Gen"].append(g_loss)
      
      
      # If at save interval => save generated image samples
      if epoch % log_inter == 0:
        display.clear_output(wait=True)
        self.generate_and_save_images(epoch)
        self.plot_loss(losses)
        print('Epoch:{} [Discriminative loss: {:.4f}],[Generative loss: {:.4f}]'.format(epoch, d_loss[0], g_loss))  
      
      inicial += batch_size
      if (inicial > len(train_data) - batch_size):
        inicial = 0  

        
    display.clear_output(wait=True)
    self.generate_and_save_images(epoch)
    self.plot_loss(losses)
      
      
  def generate_and_save_images(self, epoch):
    # training is set to False, all layers run in inference mode
    noise = np.random.normal(0, 1, (25, self.z_dim))
    predictions =  self.generator.predict(noise)

    fig = plt.figure(figsize=(5,5))

    for i in range(predictions.shape[0]):
      plt.subplot(5, 5, i+1)
      plt.imshow(predictions[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
      plt.axis('off')

    label = 'Epoch {0}'.format(epoch)
    fig.text(0.51, 0.05, label, ha='center',fontsize=14)      
    plt.savefig('NotMNIST_DCGAN_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

  def plot_loss(self, losses):
    plt.figure(figsize=(6,5))
    plt.plot(losses["Dis"], label='discrimininator loss')
    plt.plot(losses["Gen"], label='generator loss')
    plt.legend()
    plt.show()
    
  def randomize(self, dataset):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    return shuffled_dataset

In [ ]:
dcgan_4 = NotMNIST_DCGAN_4(height, width, channels, z_dim_noise)
dcgan_4.train(epochs=50000, train_data = x_train_clean, batch_size=512, log_inter=50)

In [ ]:
import glob
import imageio
with imageio.get_writer('NotMNIST.gif', mode='I') as writer:
  filenames = glob.glob('NotMNIST_DCGAN_images/image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.9)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# A hack to display the GIF inside this notebook
os.rename('NotMNIST.gif', 'NotMNIST.gif.png')

In [ ]:
display.Image(filename="NotMNIST.gif.png")

In [ ]:
!zip -r /content/file.zip /content/NotMNIST_DCGAN_images

In [ ]:
from google.colab import files
files.download("/content/file.zip")